In [44]:
from keras.models import Sequential
from keras.layers import InputLayer, Dense
import pandas as pd
import numpy as np

In [45]:
#Importing data
x_train = pd.read_csv('../pre-processamento-e-transformacao/X_TRAIN.csv')
x_test = pd.read_csv('../pre-processamento-e-transformacao/X_TEST.csv')
y_train = pd.read_csv('../pre-processamento-e-transformacao/Y_TRAIN.csv')
y_test = pd.read_csv('../pre-processamento-e-transformacao/Y_TEST.csv')
y_train_log = pd.read_csv('../pre-processamento-e-transformacao/Y_TRAIN_LOG.csv')
y_test_log = pd.read_csv('../pre-processamento-e-transformacao/Y_TEST_LOG.csv')

In [46]:
# Normalizar
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
x_train = scaler.fit_transform(x_train)
x_test  = scaler.transform(x_test)


ValueError: X has 259 features, but MinMaxScaler is expecting 283 features as input.

In [ ]:
# Creating model
input_neuron, hidden_neuron, output_neuron = 283, 3, 1

mlp = Sequential([
    InputLayer(input_neuron),
    Dense(hidden_neuron, activation="relu"),
    Dense(output_neuron)
])

In [ ]:
# Summary representation
mlp.summary()

In [ ]:
# Compiling model
mlp.compile(optimizer="adam", loss="mse")

In [ ]:
# Training the model
batch_size = 16
Log = mlp.fit(x_train, y_train, batch_size=batch_size, epochs=50)

## Medidas Desempenho

* **r2_score**(y_test, y_pred)

* **Gráfico de Dispersão**

* **median_absolute_error**(y_test, y_pred)


In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import median_absolute_error, r2_score

y_pred = mlp.predict(x_test)

print(y_test)
print(y_pred)

r2 = r2_score(y_test, y_pred)

mae = median_absolute_error(y_test, y_pred)

# Plot results
ax0 = plt.axes()

ax0.scatter(y_test, y_pred)
ax0.plot([0, 2000], [0, 2000], '--k')
ax0.set_ylabel('Target predicted')
ax0.set_xlabel('True Target')
ax0.set_title('Sem transformação logarítmica do alvo')
ax0.text(100, 1750, r'$R^2$=%.2f, MAE=%.2f' % (
    r2, mae))
ax0.set_xlim([0, 2000])
ax0.set_ylim([0, 2000])


In [ ]:
## Y log

f, (ax0, ax1) = plt.subplots(1, 2)

ax0.hist(y_train, bins=100, density=True)
ax0.set_xlim([0, 2000])
ax0.set_ylabel('Probability')
ax0.set_xlabel('Target')
ax0.set_title('Target distribution')

ax1.hist(y_train_log, bins=100, density=True)
ax1.set_ylabel('Probability')
ax1.set_xlabel('Target')
ax1.set_title('Transformed target distribution')

f.suptitle("Synthetic data", y=0.06, x=0.53)
f.tight_layout(rect=[0.05, 0.05, 0.95, 0.95])


In [ ]:
## Predicting with y log

Log = mlp.fit(x_train, y_train_log, batch_size=batch_size, epochs=50)

y_pred = np.expm1(mlp.predict(x_test))

r2 = r2_score(y_test_log, y_pred)

mae = median_absolute_error(y_test_log, y_pred)

# Plot results
ax0 = plt.axes()

ax0.scatter(y_test_log, y_pred)
ax0.plot([0, 2000], [0, 2000], '--k')
ax0.set_ylabel('Target predicted')
ax0.set_xlabel('True Target')
ax0.set_title('Sem transformação logarítmica do alvo')
ax0.text(100, 1750, r'$R^2$=%.2f, MAE=%.2f' % (
    r2, mae))
ax0.set_xlim([0, 2000])
ax0.set_ylim([0, 2000])